# AgentCore Gateway with Policy Enforcement

This notebook demonstrates:

1. Creating a gateway with OAuth and Lambda target
2. Setting up policy engine with refund limit ($1000)
3. Testing policy enforcement (direct HTTP and agent)
4. Policy Generation: Generating policies from natural language
5. Cleanup

## Setup

In [24]:
import json
import time

import boto3

from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient
from bedrock_agentcore_starter_toolkit.utils.lambda_utils import create_lambda_function

REGION = "ap-southeast-2"
REFUND_LIMIT = 1000

In [25]:
# Lambda code
refund_lambda_code = """
def lambda_handler(event, context):
    amount = event.get('amount', 0)
    return {
        "status": "success",
        "message": f"Refund of ${amount} processed successfully",
        "amount": amount
    }
"""

# Create gateway
gateway_client = GatewayClient(region_name=REGION)
cognito_response = gateway_client.create_oauth_authorizer_with_cognito("PolicyDemo")
gateway = gateway_client.create_mcp_gateway(
    name="RefundGateway", authorizer_config=cognito_response["authorizer_config"], enable_semantic_search=False
)
gateway_client.fix_iam_permissions(gateway)
time.sleep(30)

# Create Lambda
session = boto3.Session(region_name=REGION)
lambda_arn = create_lambda_function(
    session=session,
    logger=gateway_client.logger,
    function_name=f"RefundTool-{int(time.time())}",
    lambda_code=refund_lambda_code,
    runtime="python3.13",
    handler="lambda_function.lambda_handler",
    gateway_role_arn=gateway["roleArn"],
    description="Refund tool for policy demo",
)

# Add Lambda target
gateway_client.create_mcp_gateway_target(
    gateway=gateway,
    name="RefundTarget",
    target_type="lambda",
    target_payload={
        "lambdaArn": lambda_arn,
        "toolSchema": {
            "inlinePayload": [
                {
                    "name": "process_refund",
                    "description": "Process a customer refund",
                    "inputSchema": {
                        "type": "object",
                        "properties": {"amount": {"type": "integer", "description": "Refund amount in dollars"}},
                        "required": ["amount"],
                    },
                }
            ]
        },
    },
)

# Create policy
policy_client = PolicyClient(region_name=REGION)
engine = policy_client.create_or_get_policy_engine(
    name="RefundPolicyEngine", description="Policy engine for refund governance"
)

In [26]:
cedar_statement = (
    f"permit(principal, "
    f'action == AgentCore::Action::"RefundTarget___process_refund", '
    f'resource == AgentCore::Gateway::"{gateway["gatewayArn"]}") '
    f"when {{ context.input.amount < {REFUND_LIMIT} }};"
)

policy = policy_client.create_or_get_policy(
    policy_engine_id=engine["policyEngineId"],
    name="refund_limit_policy",
    description=f"Allow refunds under ${REFUND_LIMIT}",
    definition={"cedar": {"statement": cedar_statement}},
)

# Attach policy to gateway
gateway_client.update_gateway_policy_engine(
    gateway_identifier=gateway["gatewayId"], policy_engine_arn=engine["policyEngineArn"], mode="ENFORCE"
)

# Save config
config = {
    "gateway_url": gateway["gatewayUrl"],
    "gateway_id": gateway["gatewayId"],
    "gateway_arn": gateway["gatewayArn"],
    "policy_engine_id": engine["policyEngineId"],
    "client_info": cognito_response["client_info"],
    "region": REGION,
    "refund_limit": REFUND_LIMIT,
}

with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Setup complete")
print(f"Gateway: {gateway['gatewayUrl']}")
print(f"Policy: Allow refunds < ${REFUND_LIMIT}")

## Get Access Token

In [27]:
with open("config.json", "r") as f:
    config = json.load(f)

gateway_client = GatewayClient(region_name=config["region"])
access_token = gateway_client.get_access_token_for_cognito(config["client_info"])
print("✅ Access token obtained")

## Test: Direct HTTP

In [28]:
import requests


def test_refund(amount):
    response = requests.post(
        config["gateway_url"],
        headers={"Content-Type": "application/json", "Authorization": f"Bearer {access_token}"},
        json={
            "jsonrpc": "2.0",
            "id": 1,
            "method": "tools/call",
            "params": {"name": "RefundTarget___process_refund", "arguments": {"amount": amount}},
        },
    )
    return json.dumps(response.text)


print(f"Test $500: {test_refund(500)}")
print(f"Test $2000: {test_refund(2000)}")

## Test: Agent

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient


def get_tools(mcp_client):
    tools, token = [], None
    while True:
        result = mcp_client.list_tools_sync(pagination_token=token)
        tools.extend(result)
        if not result.pagination_token:
            break
        token = result.pagination_token
    return tools


model = BedrockModel(inference_profile_id="anthropic.claude-3-7-sonnet-20250219-v1:0", streaming=True)
mcp_client = MCPClient(
    lambda: streamablehttp_client(config["gateway_url"], headers={"Authorization": f"Bearer {access_token}"})
)

with mcp_client:
    agent = Agent(model=model, tools=get_tools(mcp_client))

    print("\n=== $500 Refund ===")
    print(agent("Process a refund of $500"))

    print("\n=== $2000 Refund ===")
    print(agent("Process a refund of $2000"))

    print("\n=== $500 Refund (again) ===")
    print(agent("Process a refund of $500"))

## Test: Policy Generation

Demonstrates the `generate_policy()` convenience method that generates Cedar policies from natural language.

In [31]:
print("\n" + "=" * 60)
print("Testing Policy Generation")
print("=" * 60 + "\n")

# Test the generate_policy() convenience method
policy_client = PolicyClient(region_name=config["region"])

# Example: Generate policy from natural language
natural_language_input = "Allow refunds for amounts less than $500"

print(f"📝 Natural Language Input: '{natural_language_input}'")
print(f"🎯 Target Resource: {config['gateway_arn']}")
print("\n⏳ Generating Cedar policy...\n")

try:
    # Use the new convenience method with automatic polling
    result = policy_client.generate_policy(
        policy_engine_id=config["policy_engine_id"],
        name=f"policy_gen_test_{int(time.time())}",
        resource={"arn": config["gateway_arn"]},
        content={"rawText": natural_language_input},
        fetch_assets=True,  # Automatically fetch generated policies
    )

    print("✅ Policy generation complete!\n")
    print(f"Generation ID: {result['policyGenerationId']}")
    print(f"Status: {result['status']}")

    # Display generated policies
    if "generatedPolicies" in result and result["generatedPolicies"]:
        print(f"\n📜 Generated {len(result['generatedPolicies'])} Cedar Policies:\n")
        for i, policy_asset in enumerate(result["generatedPolicies"], 1):
            definition = policy_asset.get("definition", {})
            cedar = definition.get("cedar", {})
            statement = cedar.get("statement", "N/A")
            print(f"Policy {i}:")
            print(f"{statement}")
            print()
    else:
        print("\n⚠️ No policies were generated")

except Exception as e:
    print(f"❌ Error during policy generation: {str(e)}")

print("=" * 60)

## Alternative: Create Gateway with Existing Policy

This demonstrates creating a gateway with an existing policy engine ARN using the `policy_engine_config` parameter.

In [32]:
# Alternative approach: Create gateway with existing policy engine
# This uses the policy_engine_config parameter to attach policy during gateway creation

gateway_client2 = GatewayClient(region_name=REGION)
cognito_response2 = gateway_client2.create_oauth_authorizer_with_cognito("PolicyDemo2")

# Create gateway WITH policy engine config (new feature!)
gateway2 = gateway_client2.create_mcp_gateway(
    name="RefundGateway2",
    authorizer_config=cognito_response2["authorizer_config"],
    enable_semantic_search=False,
    policy_engine_config={
        "arn": engine["policyEngineArn"],
        "mode": "ENFORCE",  # or "LOG_ONLY"
    },
)

gateway_client2.fix_iam_permissions(gateway2)
time.sleep(30)

# Add Lambda target
lambda_arn2 = create_lambda_function(
    session=session,
    logger=gateway_client2.logger,
    function_name=f"RefundTool2-{int(time.time())}",
    lambda_code=refund_lambda_code,
    runtime="python3.13",
    handler="lambda_function.lambda_handler",
    gateway_role_arn=gateway2["roleArn"],
    description="Refund tool for policy demo (gateway 2)",
)

gateway_client2.create_mcp_gateway_target(
    gateway=gateway2,
    name="RefundTarget2",
    target_type="lambda",
    target_payload={
        "lambdaArn": lambda_arn2,
        "toolSchema": {
            "inlinePayload": [
                {
                    "name": "process_refund",
                    "description": "Process a customer refund",
                    "inputSchema": {
                        "type": "object",
                        "properties": {"amount": {"type": "integer", "description": "Refund amount in dollars"}},
                        "required": ["amount"],
                    },
                }
            ]
        },
    },
)

# Update config to include second gateway
with open("config.json", "r") as f:
    config = json.load(f)

config["gateway2_url"] = gateway2["gatewayUrl"]
config["gateway2_id"] = gateway2["gatewayId"]
config["gateway2_arn"] = gateway2["gatewayArn"]
config["client_info2"] = cognito_response2["client_info"]

with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Alternative gateway created WITH policy engine config")
print(f"Gateway 2: {gateway2['gatewayUrl']}")
print("Policy already attached via policy_engine_config parameter!")

## Cleanup

In [33]:
with open("config.json", "r") as f:
    config = json.load(f)

policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])

gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(config["gateway_id"], config["client_info"])

# Cleanup second gateway if it exists
if "gateway2_id" in config:
    gateway_client.cleanup_gateway(config["gateway2_id"], config["client_info2"])
    print("✅ Gateway 2 cleanup complete")


print("✅ Cleanup complete")